In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys

import tensorflow as tf

# Prepare the data

In [ ]:
%%time

num_classes = 2

dataset = pd.read_csv('/kaggle/input/titanic/train.csv')

# Data split between train and test sets
percent_split = int((len(dataset)*75) / 100) # 75% for train and test

x_train = dataset[:percent_split]
y_train = x_train['Survived']

x_test = dataset[percent_split+1:]
y_test = x_test['Survived']

# Fix features
features = ['Pclass', 'Sex', 'SibSp', 'Parch']
x_train  = pd.get_dummies(x_train[features])
x_test   = pd.get_dummies(x_test[features])

# Fix data for the NN
x_train = np.asarray(x_train)
x_test  = np.asarray(x_test)

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

# Data for predictions
predict_data = pd.read_csv('/kaggle/input/titanic/test.csv')
x_predict    = pd.get_dummies(predict_data[features])
x_predict    = np.asarray(x_predict)

input_shape = x_train[0].shape # Input dimensions

print('\ninput shape:', input_shape)
print('\nN train elements:', x_train.shape[0])
print('N test elements   :', x_test.shape[0])
print('\none hot enconding:{}\n'.format(y_train[0, :]))

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape :', x_test.shape)
print('y_test shape :', y_test.shape)

# Build the model

In [ ]:
model = tf.keras.Sequential()
#model.add(tf.layers.Dense(512, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(1024, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax')) # One Neuron for each class
model.summary()

# Train the model

In [ ]:
%%time

batch_size = 128
epochs     = 32

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# Evaluate the trained model

In [ ]:
%%time

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss    :', score[0])
print('Test accuracy:', score[1], '\n')

predictions = np.argmax(model.predict(x_predict), axis=-1)

output = pd.DataFrame({'PassengerId': predict_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")